locate tokens with highest gradient norm. Is the sign important? 
- if gradient is positive: having a value i
- if gradient is negative: having a value in that 

In [1]:
import os
print(os.getcwd())
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import torch
from torch.nn import Softmax
from torch.utils.data import DataLoader

from notebooks.energy_model_retrain.load_ckpt import define_model

import json
import argparse
import time
from itertools import chain
from tqdm import tqdm


/home/hyeryungson/mucoco


ModuleNotFoundError: No module named 'notebooks.load_ckpt_w_attn'

In [ ]:
import os
from tqdm import tqdm
import numpy as np
import torch
import torch.nn.functional as F
import pandas as pd
import csv

In [ ]:
BATCH_SIZE=64
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [4]:
# 원본 데이터 로드
samples=pd.read_csv('./notebooks/results/samples.csv')
# label이 1인 데이터만 사용
sample1=samples.loc[samples['label']==1].copy()

In [5]:
# load trained model
ckpt_path='/home/hyeryungson/mucoco/models_bak_contd/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin'
model, config, tokenizer = define_model(ckpt_path, output_attentions=True, output_hidden_states=True)

None
None
look above for padding
Adding special tokens


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

['<s>', '<pad>', '</s>', '<unk>', 'madeupword0000', 'madeupword0001', 'madeupword0002', '<mask>']
computing vecmap
torch.Size([1280, 768]) torch.Size([768, 1280])


In [6]:
# 논문에서는 gpt-2의 tokenizer를 사용하였으므로, mask token이 기존에는 없었음
tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

# tokenizer에 mask token 추가
SPECIAL_TOKENS = {"mask_token": "<mask>"}
tokenizer.add_special_tokens(SPECIAL_TOKENS)

tokenizer.all_special_ids, tokenizer.all_special_tokens, tokenizer.vocab_size

([50256, 50257], ['<|endoftext|>', '<mask>'], 50257)

In [82]:
test_sent = sample1['text'].tolist()[0:10]
batch = tokenizer(test_sent, padding=True, return_tensors="pt", truncation=True)

In [76]:
test_sent

["I'm more concerned about the mind control being practiced by facists like tRump!",
 "Right wing terror apologist. You're a sick one.",
 "a way to offset the taxpayer is to fire all the border guards.  We clearly don't need them.  They ain't enforcing anything except coddling these so called (very healthy and rich looking) refugees from the United States of America.  The joke is on us.  Sad.  Jughead should be grossly ashamed of his pathetic leadership. A sad day for Canada indeed.",
 'The man from Kenya Born in Kenya, the Republicans & Democrats let this man destroy America not one step up and tried to stop, Look at America after 7 yrs. 7 mths. of this jerk running around do nothing, make America look weak.',
 'Someone who doesn\'t know the difference between weather and climate and who thinks that the science  upon which all of this is based comes from meteorologists has no business referring to other people as "idiots."',
 "The sheep are swallowing it up. You'd have to be a complet

In [37]:
batch.input_ids[0].shape

torch.Size([196])

In [8]:
classifier_output = model.forward(
batch["input_ids"].cuda(),
attention_mask=batch["attention_mask"].cuda(),
)

In [15]:
classifier_output['logits']

tensor([[-2.8512,  3.0555],
        [-3.7042,  4.5156],
        [-3.6532,  4.4367],
        [-3.7493,  4.5800],
        [-3.6698,  4.4535],
        [-3.5594,  4.2812],
        [-3.6983,  4.4980],
        [-2.4656,  2.5589],
        [-3.5377,  4.2471],
        [-3.0818,  3.4175]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [39]:
classifier_output['hidden_states'][2].shape

torch.Size([10, 196, 768])

In [91]:
softmax=torch.nn.Softmax(dim=-1)
softmax(classifier_output.logits)

tensor([[2.7137e-03, 9.9729e-01],
        [2.6920e-04, 9.9973e-01],
        [3.0652e-04, 9.9969e-01],
        [2.4127e-04, 9.9976e-01],
        [2.9646e-04, 9.9970e-01],
        [3.9329e-04, 9.9961e-01],
        [2.7561e-04, 9.9972e-01],
        [6.5325e-03, 9.9347e-01],
        [4.1584e-04, 9.9958e-01],
        [1.5022e-03, 9.9850e-01]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [11]:
# embedding layer
for n, p in list(model.named_parameters())[:10]:
    print(n, p.shape)

roberta.embeddings.word_embeddings.0.weight torch.Size([50257, 1280])
roberta.embeddings.word_embeddings.1.weight torch.Size([768, 1280])
roberta.embeddings.position_embeddings.weight torch.Size([514, 768])
roberta.embeddings.token_type_embeddings.weight torch.Size([1, 768])
roberta.embeddings.LayerNorm.weight torch.Size([768])
roberta.embeddings.LayerNorm.bias torch.Size([768])
roberta.encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.query.bias torch.Size([768])
roberta.encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.key.bias torch.Size([768])


In [ ]:
### 768 hidden state size
### 196 number of tokens in a sequence
### 10 batch size

In [17]:
# # loss? # output itself?
# loss = F.cross_entropy(classifier_output[0], torch.ones(len(test_sent)).long().cuda())
# loss.backward()

In [41]:
layer = classifier_output['hidden_states'][0]
layer.retain_grad()
classifier_output['logits'][0][1].backward(retain_graph=True)

In [43]:
layer.grad.shape

torch.Size([10, 196, 768])

In [44]:
norm = torch.norm(layer.grad, dim=-1)
norm = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))

In [45]:
norm.shape

torch.Size([10, 196])

In [46]:
norm

tensor([[2.9708e-01, 2.2867e-01, 1.8651e-01,  ..., 1.0000e-10, 1.0000e-10,
         1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 1.0000e-10, 1.0000e-10,
         1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 1.0000e-10, 1.0000e-10,
         1.0000e-10],
        ...,
        [1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 1.0000e-10, 1.0000e-10,
         1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 1.0000e-10, 1.0000e-10,
         1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10,  ..., 1.0000e-10, 1.0000e-10,
         1.0000e-10]], device='cuda:0')

In [53]:
to_edit_tokens = torch.topk(norm, 5, dim=-1)

In [54]:
to_edit_tokens

torch.return_types.topk(
values=tensor([[1.1212e+00, 7.3591e-01, 5.1177e-01, 5.0955e-01, 5.0683e-01],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10],
        [1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10, 1.0000e-10]],
       device='cuda:0'),
indices=tensor([[11, 12,  6,  7, 13],
        [ 1,  0,  2,  4,  3],
        [ 1,  0,  2,  4,  3],
        [ 1,  0,  2,  4,  3],
        [ 1,  0,  2,  4,  3],
        [ 1,  0,  2,  4,  3],
        [ 1,  0,  2,  4,  3],
        [ 1,  0,  2,  4,

In [57]:
batch.input_ids[0][to_edit_tokens.indices[0]]

tensor([1777, 1023, 2000, 1630,  588])

In [83]:
original_text = batch.copy()

In [84]:
for i in range(len(batch.input_ids)): 
    batch.input_ids[i][to_edit_tokens.indices[i]] = 50257

In [86]:
for i in range(len(batch.input_ids)): 
    print(test_sent[i])
    print(tokenizer.decode(batch.input_ids[i]))
    print('--')

I'm more concerned about the mind control being practiced by facists like tRump!
I'm more concerned about the<mask><mask> being practiced by<mask><mask><mask> tRump!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|e

In [58]:
batch.input_ids[1][to_edit_tokens.indices[1]]

tensor([ 8539, 11028,  3246,  7451,  2471])

In [59]:
batch.input_ids[2][to_edit_tokens.indices[2]]

tensor([  835,    64,   284,   262, 11677])

In [60]:
batch.input_ids[0][to_edit_tokens.indices[0]] = 50257

In [50]:
tokenizer.decode(batch.input_ids[0])

"I'm more concerned about the mind control being practiced by facists like tRump!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endofte

In [61]:
tokenizer.decode(batch.input_ids[0])

"I'm more concerned about the<mask><mask> being practiced by<mask><mask><mask> tRump!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|end

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
np.random.seed(args.seed)
torch.manual_seed(args.seed)

K = 1
rbt_config = RobertaConfig.from_pretrained(args.rbt_path, cache_dir=None)
rbt_tknzr = RobertaTokenizer.from_pretrained(args.rbt_path, do_lower_case=False)
rbt_model = RobertaForMTL.from_pretrained(args.rbt_path, config=rbt_config,
                                            task_names=['bertscore', 'maskedlm', 'cls'])
rbt_model.to(device)
oreo = OREO(rbt_model=rbt_model, rbt_tknzr=rbt_tknzr, k=K, device=device)
data = pd.read_csv(args.infile, sep='\n', quoting=csv.QUOTE_NONE, header=None)
results = []

for orgs in tqdm(batchify(data[0].tolist(), args.batch_size), total=len(data[0])/args.batch_size, desc='loading input'):
    torch.cuda.empty_cache()
    cand_inps_sents = orgs.copy()
    edit_track = [[] for _ in range(len(orgs))]

    for step in range(args.iter_step):
        torch.cuda.empty_cache()

        inps = [rbt_tknzr.tokenize(sent, add_prefix_space=True)[:50] for sent in cand_inps_sents]

        if args.attribute == 'formality':
            abbr_pos = oreo.select_abbr_span(inps)

        cand_inps, cand_lens, _, _ = collate_no_tokenize(inps, rbt_tknzr.convert_tokens_to_ids, device=device)
        bsz, seqlen = cand_inps.size()
        oreo.model.output_hidden_states = True
        attr_val_org, hid_states_org = oreo.cal_attr(cand_inps, hook_hid_grad=True)
        oreo.model.output_hidden_states = False

        loss = F.cross_entropy(attr_val_org, torch.ones(bsz).long().to(device))
        loss.backward()

        attr_val = F.softmax(attr_val_org, dim=1).cpu()
        del attr_val_org
        attr_val_mask = torch.where(attr_val[:, 1] > args.cls_thld, torch.zeros(bsz),
                                    torch.ones(bsz)).bool().tolist()

        attr_scores = attr_val[:, 1].tolist()
        for i, (sent, attr_score) in enumerate(zip(cand_inps_sents, attr_scores)):
            ex = {'sent': sent, 'score': attr_score}
            edit_track[i].append(ex)

        ent_mask = get_entity_mask(cand_inps_sents, cand_inps, rbt_tknzr, add_prefix_space=True)
        grad_mask = (cand_inps.ne(oreo.pad_idx)) * (cand_inps.ne(oreo.bos_idx)) * \
                    (cand_inps.ne(oreo.eos_idx)) * ent_mask

        del cand_inps, ent_mask

        for i, state in enumerate(hid_states_org):
            norm = torch.norm(state.grad, dim=-1).unsqueeze(2)
            norm = torch.where(norm > 0, norm, torch.full_like(norm, 1e-10))
            if i == 0:
                max_span_len = torch.floor((cand_lens - 2) * args.max_mask_ratio)
                max_span_len = torch.where(max_span_len < 1, torch.ones_like(max_span_len), max_span_len)
                emb_ngram_top1 = get_ngram_topk(norm.squeeze(-1), grad_mask, args.C, max_span_len, args.fixed_span_len)
                break

        del grad_mask, hid_states_org
        oreo.model.zero_grad()

        if args.attribute == 'formality':
            ins_pos_l = [item if item else emb_ngram_top1[i] for i, item in enumerate(abbr_pos)]
        elif args.attribute == 'simplicity':
            ins_pos_l = emb_ngram_top1

        cand_inps, cand_lens, _, ins_pos = collate_inp_mask_after_span(inps, rbt_tknzr.convert_tokens_to_ids,
                                                                        ins_pos_l,
                                                                        device=device)

In [ ]:
def get_ngram_topk(inps_grad, mask, C, max_span_len, fixed_span_len=3):
    inps_grad = torch.where(mask.eq(1), inps_grad, torch.full_like(inps_grad, -1e16))
    bsz, seqlen = inps_grad.size()
    
    ngrams_b = [[] for _ in range(bsz)]
    for ngram in range(1, fixed_span_len + 1):
        ngram_grad = 0.
        for i in range(ngram):
            ngram_grad += inps_grad[:, i: i + (seqlen - ngram + 1)]
        
        value, position = torch.topk(ngram_grad / (ngram + C), 1, dim=1)
        for bat, (val, pos) in enumerate(zip(value.cpu().tolist(), position.cpu().tolist())):
            for v, p in zip(val, pos):
                if v > 0:
                    ngrams_b[bat].append((v, list(range(p, p + ngram))))
                else:
                    assert ValueError, "Value of topk grad cannot be negative"

    sorted_ngrams_b = []
    for bat, seq in enumerate(ngrams_b):
        seq = [val_idx for val_idx in seq if len(val_idx[1]) <= max_span_len[bat]]
        sorted_ngrams_b.append(sorted(seq, key=lambda x: x[0], reverse=True)[0][1])

    return sorted_ngrams_b